# The usage of full-saddle-point dipoles

This notebook shows a simplified way to infer the microscopic response with a simplified model based on the Full Saddle Point Approximation (FSPA), which estimates the dipole response based on semi-classical electron's trajectories. THe only input parameter of these simplified calculations is the local intensity of the driving field. As a result, the microscopic response is obtained as a simple mask of the intensity and phase profiles of the driving pulse. The final result of the notebook is a ***coherence map***, which provides an estimate of the macroscopic volume where all the microscopic emitters contribute constructively.


The outline of the calculation, followed in this notebook, is:
1) We load the data of the IR propagation.
2) We obtain the complexified form of the driving field, which gives the phase and intensity profile of the driving pulse.
3) We use an external code to compute the FSPA-dipoles for a slected harmonic order in the required intensity range.
4) We create the coherence map. 

## Load libraries

In [ ]:
## python modules used within this notebook
import numpy as np
from scipy import integrate
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.colors as colors
import os
import shutil
import h5py
import sys
import subprocess
import MMA_administration as MMA
import mynumerics as mn
import units
from IPython.display import display, Markdown
from IPython.display import HTML

import dataformat_CUPRAD as dfC
import HHG
import plot_presets as pp 

import XUV_refractive_index as XUV_index

matplotlib.rcParams['animation.embed_limit'] = 200.

%matplotlib inline

## Load data
We load two simulation that will be compared. Because we used fine grids, we use only a sparser subaarray of the data. Note that we will need a $z$-derivatives of the driving field. We, therefore, keep a fine longitudinal resolution. Next, we will need the intensity, $I(z,r,t)$, and phase, $\Phi(z,r,t)$, profiles of the driving field.

In [ ]:
# The paths
h5file1 = os.path.join('/mnt','d','sharepoint', 'OneDrive - ELI Beamlines', 'data', 'Sunrise', 'demos', 'coherence_map', 't1', 'results_map1.h5')
h5file2 = os.path.join('/mnt','d','sharepoint', 'OneDrive - ELI Beamlines', 'data', 'Sunrise', 'demos', 'coherence_map', 't2', 'results_map2.h5')

# here we specify a coarser r-grid while loading the data
r_full_resolution = [False, 10e-6/3., 180e-6] # ['use the coarser data', radial step, maximal radial coordinate] (all in SI units)

with h5py.File(h5file1,'r') as f1, h5py.File(h5file2,'r') as f2:

    # load the data
    CUPRAD_res = [dfC.get_data(f,r_resolution=r_full_resolution) for f in (f1,f2)]

    # furher pre-processings of the data
    for result, f in zip(CUPRAD_res,(f1,f2)):
        result.get_plasma(f,r_resolution=r_full_resolution)     # load plasma profiles from the file
        result.vacuum_shift()                                   # use the speed-of-light reference frame for the pulse
        result.complexify_envel(output='add')                   # add complexified electric field profiles in the main class

    Intensity_maps = [mn.FieldToIntensitySI(abs(foo.E_zrt_cmplx_envel)) for foo in CUPRAD_res] # Intensity profile I(z,r,t) (SI)
    phase_maps     = [np.angle(foo.E_zrt_cmplx_envel)                   for foo in CUPRAD_res] # Phase profile (z,r,t) (rad)

We will need $\frac{\partial I(z,\rho,t)}{\partial z}$ and $\frac{\partial \Phi(z,\rho,t)}{\partial z}$:

In [ ]:
Intensity_spatial_gradients = [np.gradient(Intensity_map,result.zgrid,axis=0,edge_order=2)
                               for Intensity_map, result in zip(Intensity_maps,CUPRAD_res)]

phase_spatial_gradients = [np.zeros(np.shape(phase_map)) for phase_map in phase_maps]
for phase_spatial_gradient, phase_map, result in zip(phase_spatial_gradients, phase_maps, CUPRAD_res):
    for k1 in range(result.Nr):
        for k2 in range(result.Nt):
            phase_spatial_gradient[:,k1,k2] = np.gradient(np.unwrap(phase_map[:,k1,k2]),result.zgrid,edge_order=2)

## Prepare FSPA calculation

**Note:** The code for computing FSPA has to be compiled in the respective folder of the multiscale model. See the `README.md` therein.

The next ingredient we need is to calculate the Saddle Points. (The atomic units are used in the whole FSPA section and the calculation as we moved to the microscopic description.) The microscopic response is given (within the SFA framework):
$$ d(\omega) = -\frac{i}{\sqrt{2\pi}} \int_{-\infty}^{+\infty} dt \left( \int_0^t dt' \int d^3k \left[ \mathcal{E}(t') \cdot \mathbf{d}\left(\mathbf{k} + \mathbf{A}(t')\right) e^{i(\omega t - \mathcal{S}(t',t,\mathbf{k}))} \right] d^*\left(\mathbf{k} + \mathbf{A}(t)\right) \right) dt + c.c. \,,$$
where $\mathcal{S}(t', t, \mathbf{k}) = \int_{t'}^{t} \left( \frac{\left( \mathbf{k} + \mathbf{A}(t'') \right)^2}{2} - E_g \right) dt''$ is the semi-classical action for the electron ejected from the ground state with the energy $E_g$. The driving field is represented by the vector potential $\bm{A}$ (linked with the electric field $\bm{\mathcal{E}} = -\partial_t \bm{A}$). $k$ denotes the integration over the momentum space.
The Saddle point are performed over all the integration variables resulting to
$$ d_{\text{S-P}}(\omega) \approx - \frac{i}{\sqrt{2\pi}} \left( \frac{-2\pi i}{\sqrt{\det \Phi''_\omega}} \right)^{\frac{5}{2}} \left[ \mathcal{E}(t_i) \cdot \mathbf{d}\left( \mathbf{k}_{tr,t_i}^{(0)} + \mathbf{A}(t_i) \right) e^{i \Phi_\omega (t_r,t_i, \mathbf{k}_{tr,t_i}^{(0)})} \right] \mathbf{d}^*\left( \mathbf{k}_{tr,t_i}^{(0)} + \mathbf{A}(t_r) \right) + c.c. \,,
 $$
where $\Phi_\omega = \omega t - \mathcal{S}(t',t,\mathbf{k})$ is the phase providing the Saddle Points. Note that complex times are required to satisfy the equation. $\Phi''_\omega$ denotes the Hessian matrix of $\Phi_\omega$ with respect to all the integration variables ($t$, $t'$, $k$).

An important property is that the solution reflects the generation regime: there are two semiclassical trajectories (short and long) for harmonics generated in the plateau. There is only one physical solution if the harmonic is generated in the cut-off regime. It turns out that the dipole can be locally well-approximated by
$$ d_{\text{S-P}} \approx |I_{\mathrm{IR}}|^{q_{\mathrm{eff}}/2} \mathrm{e}^{-\bf{i}\alpha I_{\mathrm{IR}}} \,,$$
where $q_{\mathrm{eff}}$ and $\alpha$ are phenomenological trajectory-dpenedent coefficients.

We use an external Fortran routine that implements all the calculation, it provides us directly the dipole based on the amplitude of the input field. The input of this routine are:
* *the ionization potential of the target* $I_p$,
* *the fundamental frequency of the driving field* $\omega_0$,
* *the harmonic order of the interest* $H$,
* *the linespace of the amplitudes of the vector potential* (defined by the minimal value $A_{\mathrm{min}}$, $\Delta A$ and the number of steps $N_{\mathrm{pts}}$).

The code computes the saddle points and provide the complex dipoles $d_{\mathrm{S-P}}$ for the different trajectories. We then find the values $\alpha$ by differentiating the phase with respect to $I_{\mathrm{IR}}$.



In [ ]:
Horder =  17 # The harmonic order of the interest



## Prepare the calculation
# note we assume the fundamental quantities (driver's frequency, Ip are the same for all the simulations)

# Convert the frequency to atomic units
omega0_au = mn.ConvertPhoton(CUPRAD_res[0].omega0,'omegaSI','omegaau')
# Convert the ionization potential into the atomic units
Ip_au     = mn.ConvertPhoton(CUPRAD_res[0].Ip_eV,'eV','omegaau')

# minimal vector potential for the calculation
A0_min = 0.1

# maximal vector potential obtained from the maximal intensity in the entry plane (increased by 20 % for safety)
A0_max = 1.2*(np.sqrt(np.max([result.Intensity_entry for result in CUPRAD_res])/units.INTENSITYau))/omega0_au

# the step in the vector potential
dA = 0.01

# the number of points to span the interval [A0_min, A0_max]
N_pts = int(np.round((A0_max-A0_min)/dA)+1)



## Prepare the external calculation
mydir = os.path.join('/mnt','d','git','MMA-interactive')        # select the root directory, where we run the code (We need to process temporary files)
temporary_dir = 'temporary_FSPA_dir'                            # directory for the temporary files
temporary_dir = os.path.join(mydir,temporary_dir)
if os.path.exists(temporary_dir): shutil.rmtree(temporary_dir)  # clean the temporary directory if it existed
os.makedirs(temporary_dir)
os.chdir(temporary_dir)                                         # go to the temporary directory


## Run the calculation
# prepare the input file
with open('param.inp','w') as f:
    f.write('\n'.join([str(foo) for foo in [Ip_au,omega0_au,Horder,A0_min,dA,N_pts]]))
log_FSPA_run = subprocess.run(os.environ['FSPA_PATH']+'/FSPA.e',capture_output=True)

# load the results for the two classes of the trajectories
long_trajecotry_phase  = np.loadtxt('phase_long.dat')
short_trajecotry_phase  = np.loadtxt('phase_short.dat')

# clean the temporary directory
shutil.rmtree(temporary_dir)

## We pick only some of the outputs:
# Intensity grid   
Igrid =  long_trajecotry_phase[:,0]

# Phase
phase_long =  long_trajecotry_phase[:,7]
phase_short =  short_trajecotry_phase[:,7]

# Differentiate the phases to get (d phase / d I)
dI_phase_long = np.gradient(long_trajecotry_phase[:,7],Igrid,edge_order=2)
dI_phase_short = np.gradient(short_trajecotry_phase[:,7],Igrid,edge_order=2)


plt.figure(figsize=(4, 3)) 

# Plot the data
plt.plot(Igrid, dI_phase_short, label='short trajectory')
plt.plot(Igrid, dI_phase_long, label='long trajectory')
plt.xlim((0, 0.002))
plt.ylim((-10000., 5000.))

# Add title and labels
plt.title(r'Dipole phase derivatives')
plt.xlabel(r'$I$ [a.u.]')
plt.ylabel(r'$\frac{\partial \phi}{\partial I}$ [rad/a. u.]')
plt.legend()

# Display the plot
plt.show()

Now we have the derivatives of the phases. However, there is an issue that the calculation, the solution for the short trajectories does not converge well in the cut-off region (where the two families of trajectories merge as shown above). We just link smoothly traverse this region with an ploynomial for our purposes.

Note: Strictly speaking, the Saddle points are not valid near the cuto-off region. We can expect the interference of different quantum paths there. We will just interpolate since it is satisfactory for an overall picture. One might need to use more advanced physics in particular cases (e.g. running full TDSE, using a different SFA model, ...).

In [ ]:
from numpy.polynomial import Polynomial

# Select intensity points used for the smooth transition through the problematic region
I_points = [0.0004,0.0005,0.0015,0.002]

# Get the indices of the respective grid
points_indexes = mn.FindInterval(Igrid,I_points)

# Construct the interploating polynomial
p = Polynomial.fit(I_points,dI_phase_short[mn.FindInterval(Igrid,I_points)],3)

# Do the interpolation in the region
dI_phase_short[points_indexes[0]:points_indexes[-1]] = p(Igrid[points_indexes[0]:points_indexes[-1]])

# Plot the outcome
plt.figure(figsize=(4, 3)) 
plt.plot(Igrid, dI_phase_short, label='short trajectory')
plt.plot(Igrid, dI_phase_long, label='long trajectory')
plt.xlim((0, 0.002))
plt.ylim((-10000., 5000.))

plt.title(r'Dipole phase derivatives (smoothened)')
plt.xlabel(r'$I$ [a.u.]')
plt.ylabel(r'$\frac{\partial \phi}{\partial I}$ [rad/a. u.]')
plt.legend()

plt.show()


## Intensity profile & plasma densities
Before we proceed to the final coherence maps, we show the profile of the beam and the plasma density after the passage of the pulse to get a general overview of the generating condition. We can either inspect the maximum intensity reached by the pulse or at a fixed time. The intensity is measured using the cut-off law.

In [ ]:
result_number   = 0     # choose the result that will is plotted
plot_t_plane    = False # plot at given t_fix, plot the maximum otherwise
t_fix_relative  = 0.0   # the position of the plane for plotting relative to the "pulse duration"/2


## Plotting procedure
# Select result handle
result = CUPRAD_res[result_number]
Intensity_map = Intensity_maps[result_number]



fig = plt.figure(figsize=(14, 5.5))
ax1 = plt.subplot2grid((1, 2), (0, 0))
ax2 = plt.subplot2grid((1, 2), (0, 1))

## Plot the plasma density
# Symmetrize the data 
symmetric_y, symmetric_data =  mn.symmetrize_y(result.plasma.rgrid,result.plasma.value_zrt[:,:,-1]/result.effective_neutral_particle_density)
pc1 = ax1.pcolormesh(1e3*result.plasma.zgrid, 1e6*symmetric_y, 1e2*symmetric_data.T, shading = 'auto')
cbar1 = fig.colorbar(pc1, ax=ax1, orientation = 'horizontal')
ax1.set_ylabel(r'$\rho~[\mu \mathrm{m}]$')
ax1.set_xlabel(r'$z~[\mathrm{mm}]$')
cbar1.ax.set_xlabel('relative plasma density [%]')

## Plot the intensity profile
symmetric_y, symmetric_data =  mn.symmetrize_y(result.rgrid,
                                    HHG.ComputeCutoff(
                                        # Select if the t_fix or maximum is used
                                        Intensity_map[:,:,
                                                      mn.FindInterval(result.tgrid,t_fix_relative*result.pulse_duration_entry)
                                                    ]/units.INTENSITYau if plot_t_plane
                                        else np.max(Intensity_map, axis =2)/units.INTENSITYau,
                                        mn.ConvertPhoton(result.omega0,'omegaSI','omegaau'),
                                        mn.ConvertPhoton(result.Ip_eV,'eV','omegaau')
                                    )[1]
                                )
pc2 = ax2.pcolormesh(1e3*result.plasma.zgrid, 1e6*symmetric_y, symmetric_data.T, shading = 'auto')
# Get contours between the ionization threshold and the maximum intensity
levels = np.arange( 2*(np.ceil(mn.ConvertPhoton(result.Ip_eV,'eV','omegaau')/omega0_au))//2 + 1,
                    np.max(symmetric_data),
                    2 )
# Add contours
contour = ax2.contour(1e3*result.plasma.zgrid, 1e6*symmetric_y, symmetric_data.T, levels=levels, colors='black', linewidths=1.)
cbar2 = fig.colorbar(pc2, ax=ax2, orientation = 'horizontal')
cbar2.add_lines(contour)
ax2.set_xlabel(r'$z~[\mathrm{mm}]$')
ax2.yaxis.set_visible(False)
cbar2.ax.set_xlabel('intensity [harmonic order]')

fig.tight_layout()
plt.show()


## Coherence maps
Now, we can proceed to the coherence maps, which provide insight into phase-matching. The physical explanation of the phase-matching arises from the dephasing
$$ \Delta k_q = \Delta k_{\text{disp.}} + \Delta k_{\text{plasma}} + \Delta k_{\text{geom.}} + \Delta k_{\text{atom}} \,, $$
where $\Delta k_{\text{disp.}}$ denotes the mismatch due to the dispersion on neutrals, $\Delta k_{\text{plasma}}$ the mismatch due to the free electrons created by the ionization, $\Delta k_{\text{geom.}}$ due to the additional geometrical phase of the beam (e.g. the Gouy phase in the case of a Gaussian beam) and $\Delta k_{\text{atom}}$ the dephasing coming from the microscopic dipoles.

This model is very useful for the insight in the underlying physics. Once the field profile $\mathcal{E}(z,\rho,t)$ is known, the dephasing corresponds to the $z$-gradient of the phase $\frac{\partial \Phi}{\partial z}$, which we already computed. As we used the vacuum reference frame from the driver, the dephasing due to the XUV dispersion has to be added. Last, the microscopic phase is, naturally, not included in the driver. Using the phenomenological dipole $d_{\mathrm{S-P}} \sim \mathrm{e}^{-\alpha I_{\mathrm{IR}}}$, we obtain the dephasing as the $z$-gradient of the phase
$$ \frac{\partial}{\partial z} (-\alpha I_\mathrm{IR}) = -\alpha  \frac{\partial I_\mathrm{IR}}{\partial z} \,. $$
It means that the dephasing is given by the $\alpha$ value (obtained from the FSPA model) and the spatial intensity gradient, which we already prepared.

We can now put all three ($\frac{\partial \Phi}{\partial z}$, XUV dispersion and the dipole phase) together and reformulate the result in the words of the coherence length
$$ L_{\mathrm{coh}} = \frac{\pi}{|\Delta k_q|} \,, $$
whcih has a clear physical explanantion. If it is constant, $L_{\mathrm{coh}}$ gives exactly the spatial dimension of a volume where all the microscopic emitters sum constructively ($L_{\mathrm{coh}} = +\infty$ corroseponds to the perfect phase-matching). Although $L_{\mathrm{coh}}$ is a local quantity, it still provides a good estimate if the macroscopic phase-matching is efficient.

Here we show the coherence map in the whole medium for a different times in the pulse.

In [ ]:
### Set the plots
t_span = 0.75*1e-15*CUPRAD_res[0].pulse_duration_entry*np.array((-1.,1.)) # timespan for the plots
N_plot_anims = 20                                                       # number of the points in the span
L_coh_max_plot = 0.015                                                  # the range for the coherence map plot

ani_outpath = os.path.join('/mnt','d', 'git', 'MMA-interactive', 'export')

### Compute additional dephasings:

## obtain the dephasing due to the XUV dispersion
dispersion_tables = 'NIST'

nXUV = [XUV_index.nXUV(Horder*result.omega0, result.gas_type+'_'+dispersion_tables, 1e-3*result.pressure_mbar, complex=False)
        for result in CUPRAD_res]
delta_k_XUV = [result.k0_wave*(nXUV_-1.) for result, nXUV_ in zip(CUPRAD_res,nXUV)]


## obtain the dephasings from the FSPA dipoles
alpha_maps     = [-np.interp(Intensity_map/units.INTENSITYau,Igrid,dI_phase_short)
                  for Intensity_map in Intensity_maps] # a.u.
delta_k_dipole = [Intensity_spatial_gradient * alpha_map/units.INTENSITYau
                  for Intensity_spatial_gradient, alpha_map in zip(Intensity_spatial_gradients, alpha_maps)] # SI

## Full dephasing maps
# note the contributions except the microscopic phase are multiplied by the harmonic order to adjust to the proper frequency
delta_k_maps = [Horder*(phase_spatial_gradient + delta_k_XUV_) + delta_k_dipole_
                for phase_spatial_gradient, delta_k_XUV_, delta_k_dipole_ in
                zip(phase_spatial_gradients, delta_k_XUV, delta_k_dipole)]

In [ ]:
## Plot the maps for the first two simulations
# set the indices to choose the plots
kt_indices = [mn.FindInterval(result.tgrid,t_span) for result in CUPRAD_res]
kt_step =    [len(CUPRAD_res[k1].tgrid[kt_indices[k1][0]:kt_indices[k1][-1]])//N_plot_anims for k1 in range(len(CUPRAD_res))]
L_coh_max_plots = len(CUPRAD_res)*[L_coh_max_plot]

fig = plt.figure(figsize=(14, 5.5))
ax = [plt.subplot2grid((1, 2), (0, 0)),  # Left
      plt.subplot2grid((1, 2), (0, 1))]  # Right

pc = []; cbar =[]
for k1 in range(2): # Do the plot only for the first two simulations (easy to generalize for more)
    symmetric_y, symmetric_data =  mn.symmetrize_y(CUPRAD_res[k1].rgrid,1e3*np.pi/np.abs(delta_k_maps[k1][:,:,kt_indices[k1][0]]))
    pc.append(ax[k1].pcolormesh(1e3*CUPRAD_res[k1].zgrid, 1e6*symmetric_y, symmetric_data.T, shading = 'auto'))
    pc[k1].set_clim(0., 1e3*L_coh_max_plots[k1])
    cbar.append(fig.colorbar(pc[k1], ax=ax[k1], orientation = 'horizontal'))
    ax[k1].set_xlabel(r'$z~[\mathrm{mm}]$')
    ax[k1].set_title(r'pre-ionization: $\eta =$'+CUPRAD_res[k1].preionisation_string)
    cbar[k1].ax.set_xlabel(r'$L_{\mathrm{coh}}$ [mm]')
    
ax[0].set_ylabel(r'$\rho~[\mathrm{\mu m}]$')
ax[1].yaxis.set_visible(False)

title = fig.suptitle(r"$t={:.2f}".format(1e15*CUPRAD_res[k1].tgrid[kt_indices[0][0]]) + r'$ fs')


def update(frame):
    for k1 in range(2):

        # Update the data
        data = (mn.symmetrize_y(CUPRAD_res[k1].rgrid,1e3*np.pi/np.abs(delta_k_maps[k1][:,:,kt_indices[k1][0]+frame*kt_step[k1]]))[1]).T
        pc[k1].set_array(data.ravel())
        title.set_text(r"$t={:.2f}".format(1e15*CUPRAD_res[k1].tgrid[kt_indices[0][0]+frame*kt_step[k1]]) + r'$ fs')
    return pc


fig.tight_layout()
ani = matplotlib.animation.FuncAnimation(fig, update, frames=N_plot_anims, blit=True)
HTML(ani.to_jshtml())

The coherence map provides a good overview. However, it is not directly clear if the intensity is high enough at the given point for $H$ to be generated. A possible way to get this information is to mask the coherence map by the intensity:

In [ ]:
## Redo the plots with the masking
Horder_thershold = 2. # the number of harmonic orders below the selcted harmonic that are not masked out

# THe same figure, just the masking is included
fig = plt.figure(figsize=(14, 5.5))
ax = [plt.subplot2grid((1, 2), (0, 0)),  # Left
      plt.subplot2grid((1, 2), (0, 1))]  # Right

pc = []; cbar =[]
for k1 in range(2): # Do the plot only for the first two simulations (easy to generalize for more)
    symmetric_y, symmetric_data =  mn.symmetrize_y(CUPRAD_res[k1].rgrid,1e3*np.pi/np.abs(delta_k_maps[k1][:,:,kt_indices[k1][0]]))
    Cutoff_map_sym =  mn.symmetrize_y(CUPRAD_res[k1].rgrid, HHG.ComputeCutoff(Intensity_maps[k1][:,:,kt_indices[k1][0]]/units.INTENSITYau, omega0_au, Ip_au)[1])[1]
    masked_data = np.ma.masked_where(Cutoff_map_sym.T <= Horder-Horder_thershold, symmetric_data.T)
    pc.append(ax[k1].pcolormesh(1e3*CUPRAD_res[k1].zgrid, 1e6*symmetric_y, masked_data, shading = 'auto'))
    pc[k1].set_clim(0., 1e3*L_coh_max_plots[k1])
    cbar.append(fig.colorbar(pc[k1], ax=ax[k1], orientation = 'horizontal'))
    ax[k1].set_xlabel(r'$z~[\mathrm{mm}]$')
    ax[k1].set_title(r'pre-ionization: $\eta =$'+CUPRAD_res[k1].preionisation_string)
    cbar[k1].ax.set_xlabel(r'$L_{\mathrm{coh}}$ [mm]')

ax[0].set_ylabel(r'$\rho~[\mathrm{\mu m}]$')
ax[1].yaxis.set_visible(False)

title = fig.suptitle(r"$t={:.2f}".format(1e15*CUPRAD_res[k1].tgrid[kt_indices[0][0]]) + r'$ fs')


def update(frame):
    for k1 in range(2):

        # Update the data
        data = (mn.symmetrize_y(CUPRAD_res[k1].rgrid,1e3*np.pi/np.abs(delta_k_maps[k1][:,:,kt_indices[k1][0]+frame*kt_step[k1]]))[1]).T
        Cutoff_map_sym =  mn.symmetrize_y(CUPRAD_res[k1].rgrid, HHG.ComputeCutoff(Intensity_maps[k1][:,:,kt_indices[k1][0]+frame*kt_step[k1]]/units.INTENSITYau, omega0_au, Ip_au)[1])[1]
        masked_data = np.ma.masked_where(Cutoff_map_sym.T <= Horder-Horder_thershold, data)
        pc[k1].set_array(masked_data.ravel())
        title.set_text(r"$t={:.2f}".format(1e15*CUPRAD_res[k1].tgrid[kt_indices[0][0]+frame*kt_step[k1]]) + r'$ fs')
    return pc


fig.tight_layout()
ani = matplotlib.animation.FuncAnimation(fig, update, frames=N_plot_anims, blit=True)
HTML(ani.to_jshtml())

# Define the writer using ffmpeg for mp4 format and save it
Writer = matplotlib.animation.writers['ffmpeg']
writer = Writer(fps=3, metadata=dict(artist='Your Name'), bitrate=1800)

ani.save(os.path.join(ani_outpath,'coherence_maps.mp4'), writer=writer)

In [ ]:
## map



Cutoff_map_sym =  mn.symmetrize_y(CUPRAD_res1.rgrid, HHG.ComputeCutoff(Intensity_map1[:,:,kt_indices[0]]/units.INTENSITYau, omega0_au, Ip_au)[1])[1]

masked_data = np.ma.masked_where(Cutoff_map_sym.T <= 15, symmetric_data.T)

pc2 = ax2.pcolormesh(1e3*CUPRAD_res1.zgrid, symmetric_y, masked_data, shading = 'auto')
pc2.set_clim(0., 0.015)
# ax2.set_ylim(-rlim,rlim)
cbar2 = fig.colorbar(pc2, ax=ax2, orientation = 'horizontal')




ax1.set_ylabel(r'$\rho~[\mu \mathrm{m}]$')
ax1.set_xlabel(r'$z~[\mathrm{mm}]$')
# ax2.set_xlabel(r'$z~[\mathrm{mm}]$')

# cbar1.ax.set_xlabel('plasma density $[\mathrm{m}^{-3}]$')
# cbar2.ax.set_xlabel('relative plasma density [%]')



def update(frame):
    # Update the data
    data1 = (mn.symmetrize_y(CUPRAD_res1.rgrid,np.pi/np.abs(delta_k_map1[:,:,kt_indices[0]+frame*kt_step]))[1]).T

    pc1.set_array(data1.ravel())
    # pc1.set_clim(data1.min(), data1.max())


    Cutoff_map_sym =  mn.symmetrize_y(CUPRAD_res1.rgrid, HHG.ComputeCutoff(Intensity_map1[:,:,kt_indices[0]+frame*kt_step]/units.INTENSITYau, omega0_au, Ip_au)[1])[1]

    masked_data = np.ma.masked_where(Cutoff_map_sym.T <= 15, data1)

    pc2.set_array(masked_data.ravel())


    # title.set_text("z={:.2f}".format(1e3*zgrid_Hankel[frame+1]) + ' mm')


    return [pc1,pc2]


fig.tight_layout()
ani = matplotlib.animation.FuncAnimation(fig, update, frames=N_plot_anims, blit=True)
HTML(ani.to_jshtml())

# plt.show()

In [ ]:
# ## map
# t_span = 0.75*1e-15*CUPRAD_res1.pulse_duration_entry*np.array((-1.,1.))
# N_plot_anims = 20

# kt_indices = mn.FindInterval(CUPRAD_res1.tgrid,t_span)
# kt_step = len(CUPRAD_res1.tgrid[kt_indices[0]:kt_indices[-1]])//N_plot_anims


# delta_k_map1 = Horder*(phase_spatial_gradients1 + CUPRAD_res1.k0_wave*(nXUV-1.)) + delta_k_dipole1
# delta_k_map2 = Horder*(phase_spatial_gradients2 + CUPRAD_res2.k0_wave*(nXUV-1.)) + delta_k_dipole2

# # Cutoff_map = HHG.ComputeCutoff(Intensity_map1/units.INTENSITYau, omega0_au, Ip_au)[1] 
# # Harmonic_mask = HHG.ComputeCutoff(Intensity_map1/units.INTENSITYau, omega0_au, Ip_au)[1] 
# # Harmonic_mask[Harmonic_mask < Horder-2.] = np.nan 
# # Harmonic_mask = np.ma.masked_where(np.isnan(Harmonic_mask),Harmonic_mask) 



# fig = plt.figure(figsize=(14, 5.5))

# # Define subplots using subplot2grid
# ax1 = plt.subplot2grid((1, 2), (0, 0))  # Upper left
# ax2 = plt.subplot2grid((1, 2), (0, 1))  # Upper right


# symmetric_y, symmetric_data =  mn.symmetrize_y(CUPRAD_res2.rgrid,np.pi/np.abs(delta_k_map2[:,:,kt_indices[0]]))
# pc1 = ax1.pcolormesh(1e3*CUPRAD_res2.zgrid, symmetric_y, symmetric_data.T, shading = 'auto')
# pc1.set_clim(0., 0.015)
# # ax1.set_ylim(-rlim,rlim)
# cbar1 = fig.colorbar(pc1, ax=ax1, orientation = 'horizontal')


# Cutoff_map_sym =  mn.symmetrize_y(CUPRAD_res1.rgrid, HHG.ComputeCutoff(Intensity_map1[:,:,kt_indices[0]]/units.INTENSITYau, omega0_au, Ip_au)[1])[1]

# masked_data = np.ma.masked_where(Cutoff_map_sym.T <= 15, symmetric_data.T)

# pc2 = ax2.pcolormesh(1e3*CUPRAD_res1.zgrid, symmetric_y, masked_data, shading = 'auto')
# pc2.set_clim(0., 0.015)
# # ax2.set_ylim(-rlim,rlim)
# cbar2 = fig.colorbar(pc2, ax=ax2, orientation = 'horizontal')




# ax1.set_ylabel(r'$\rho~[\mu \mathrm{m}]$')
# ax1.set_xlabel(r'$z~[\mathrm{mm}]$')
# # ax2.set_xlabel(r'$z~[\mathrm{mm}]$')

# # cbar1.ax.set_xlabel('plasma density $[\mathrm{m}^{-3}]$')
# # cbar2.ax.set_xlabel('relative plasma density [%]')



# def update(frame):
#     # Update the data
#     data1 = (mn.symmetrize_y(CUPRAD_res2.rgrid,np.pi/np.abs(delta_k_map2[:,:,kt_indices[0]+frame*kt_step]))[1]).T

#     pc1.set_array(data1.ravel())
#     # pc1.set_clim(data1.min(), data1.max())


#     Cutoff_map_sym =  mn.symmetrize_y(CUPRAD_res2.rgrid, HHG.ComputeCutoff(Intensity_map2[:,:,kt_indices[0]+frame*kt_step]/units.INTENSITYau, omega0_au, Ip_au)[1])[1]

#     masked_data = np.ma.masked_where(Cutoff_map_sym.T <= 15, data1)

#     pc2.set_array(masked_data.ravel())
#     # pc2.set_clim(data1.min(), data1.max())    

#     # # Update the colors
#     # pc2.set_array(data.ravel())
#     # # pc.set_clim(data.min(), data.max())
#     # pc2.set_clim((10**(-orders_to_plot))*data.max(), data.max())
#     # # cbar.update_normal(pc2)

#     # title.set_text("z={:.2f}".format(1e3*zgrid_Hankel[frame+1]) + ' mm')


#     return [pc1,pc2]


# fig.tight_layout()
# ani = matplotlib.animation.FuncAnimation(fig, update, frames=N_plot_anims, blit=True)
# HTML(ani.to_jshtml())

# # plt.show()

## Plot the propagating pulse
We choose the time-and-space window to see the pulse as it propagates through the medium. Note that we measure the intensity by the "expected harmonic cutoff", these units are obtained by the formula $E_{\text{cut-off}} = I_P + 3.17U_p$ (it is directly proportional since $U_p$ is linearly proportional to the intensity). Then we plot the plasma channel create by the passage of the pulse. We show both absolute density of free electrons and also relative to the local density.

There are more technical details about the data: We plot the pulse directly as it is stored in the file. This means that we a co-moving frame defined by the group velocity, $v_g$, of the pulse: this is the computational window of CUPRAD. The group velocity $v_g$ is defined from the linear dispersion relation and depends on the chosen reference pressure and central wavelength. Possible density modulation is relative to this reference pressure, whcih is the reason we use the average pressure in our examples. Physically speaking, $v_g$ is arbitrary and needs to be considered in further processing. For example, the Pyrhonic class represented by `CUPRAD_res` contains methods to adjust to the reference given by the speed of light (both activelly by changing the data or just by sychronising the clocks in the $t$-grid).

In [ ]:

# tlim = np.asarray([t_plot_span*CUPRAD_res.pulse_duration_entry for t_plot_span in (-2. , 2.)])
# tlim2 = np.asarray([t_plot_span*CUPRAD_res2.pulse_duration_entry for t_plot_span in (-2. , 2.)])
# # rlim = 600

In [ ]:
# # Code to generate the animated figure

# k_t_min, k_t_max = mn.FindInterval(1e15*CUPRAD_res.tgrid,1.05*tlim)
# k_t_min2, k_t_max2 = mn.FindInterval(1e15*CUPRAD_res2.tgrid,1.05*tlim2)

# # k_r_max          = mn.FindInterval(1e6*CUPRAD_res.rgrid ,1.05*rlim)

# fig = plt.figure(figsize=(14, 5.5))


# ax1 = plt.subplot2grid((1, 2), (0, 0))  # Upper left
# ax2 = plt.subplot2grid((1, 2), (0, 1))  # Upper right

# # r_grid, sym_data = mn.symmetrize_y(1e6*CUPRAD_res.rgrid[:k_r_max],
# #                     (
# #                     HHG.ComputeCutoff(
# #                         mn.FieldToIntensitySI(CUPRAD_res.E_zrt[0,:k_r_max,k_t_min:k_t_max])/units.INTENSITYau,
# #                         mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','omegaau'),
# #                         mn.ConvertPhoton(CUPRAD_res.Ip_eV,'eV','omegaau')
# #                     )[1]
# #                     ).T)

# pc1 = ax1.pcolormesh(1e15*CUPRAD_res.tgrid[k_t_min:k_t_max], 1e6*CUPRAD_res.rgrid, CUPRAD_res.E_zrt[0,:,k_t_min:k_t_max], shading='auto', cmap = 'seismic')
# pc2 = ax2.pcolormesh(1e15*CUPRAD_res2.tgrid[k_t_min2:k_t_max2], 1e6*CUPRAD_res2.rgrid, CUPRAD_res2.E_zrt[0,:,k_t_min2:k_t_max2], shading='auto', cmap = 'seismic')

# ax1.set_xlim(tlim)
# # ax1.set_ylim((-rlim,rlim))

# ax1.set_title("z={:.2f}".format(1e3*CUPRAD_res.zgrid[0]) + ' mm')
# ax1.set_xlabel(r'$t~[\mathrm{fs}]$')
# ax1.set_ylabel(r'$\rho~[\mu\mathrm{m}]$')

# cbar1 = fig.colorbar(pc1, ax=ax1)
# cbar2 = fig.colorbar(pc2, ax=ax2)
# # cbar.ax.set_ylabel(r'Intensity [harmonic cut-off]', rotation=90)




# def update(frame):
#     # Update the data
#     data1 = CUPRAD_res.E_zrt[frame,:,k_t_min:k_t_max]
#     data2 = CUPRAD_res2.E_zrt[frame,:,k_t_min2:k_t_max2]
    
#     # Update the colors
#     pc1.set_array(data1.ravel())
#     pc1.set_clim(data1.min(), data1.max())
#     cbar1.update_normal(pc1)

#     pc2.set_array(data2.ravel())
#     pc2.set_clim(data2.min(), data2.max())
#     cbar2.update_normal(pc2)

#     ax1.set_title("z={:.2f}".format(1e3*CUPRAD_res.zgrid[frame]) + ' mm')

#     # Update the progress indicator
#     # progress_line.set_data([1e3*CUPRAD_res.zgrid[frame], 1e3*CUPRAD_res.zgrid[frame]],
#     #                         [CUPRAD_res.density_mod_profile_mbar.min(), CUPRAD_res.density_mod_profile_mbar.max()])

#     return [pc1,pc2]

# # Ensure the layout does not have overlaps and everything is nicely spaced
# fig.tight_layout() 

# ani = matplotlib.animation.FuncAnimation(fig, update, frames=len(CUPRAD_res.zgrid), blit=True)

# HTML(ani.to_jshtml())